In [1]:
import sys
import os
import numpy as np

from codingTools import *


In [2]:
######LOAD RAW IMAGE######
#dtype --> > stands for big endian, i stands for integer, 2 stands for 2 bytes
#originalFile="../images/03508649.1_512_512_2_0_12_0_0_0d.raw" #int16
originalFile="../images/03508649.1_128_128_1_0_8_0_0_0.raw"
#originalFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0.raw" #uint8
#ImageRaw = load_image_raw(originalFile, 1, 512, 512, "int16") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
ImageRaw = load_image_raw(originalFile, 1, 128, 128, "uint8")
#ImageRaw = load_image_raw(originalFile, 1, 2560, 2048, "uint8") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
original_image = ImageRaw.load_image_raw()
##############################

In [3]:
########STATISTICS##########
statistics_data = statistics(original_image)
entropy = statistics_data.compute_entropy()
print("order zero entropy of the image: ",entropy)

order zero entropy of the image:  6.7004541529645625


In [4]:
print(original_image)

[[[16 21 25 ... 25 22 19]
  [19 25 29 ... 29 25 22]
  [22 28 34 ... 34 29 25]
  ...
  [22 28 33 ... 34 30 26]
  [19 24 29 ... 30 25 23]
  [16 21 25 ... 25 22 19]]]


In [5]:
######SAVE RAW IMAGE######
# decodedFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0_copy.raw"
# ImageRawDecoded = save_image_raw(decodedFile)
# ImageRawDecoded.save_image_raw(original_image,"uint8")
###########

In [6]:
### Quantize ###
quantize = Quantizer(1, 10)
image_data_quantized = ImageRaw.get_image_structure_empty()
quantize.quantize(original_image, image_data_quantized)
quant_statics = statistics(image_data_quantized).compute_entropy()
print(f"Entropia: {quant_statics}")


Check Quantize: [[[1. 2. 2. ... 2. 2. 1.]
  [1. 2. 2. ... 2. 2. 2.]
  [2. 2. 3. ... 3. 2. 2.]
  ...
  [2. 2. 3. ... 3. 3. 2.]
  [1. 2. 2. ... 3. 2. 2.]
  [1. 2. 2. ... 2. 2. 1.]]]
Entropia: 3.4368520266679963


In [7]:
### Dequantize ###
dequantize = Quantizer(1, 10)
image_data_dequantized = ImageRaw.get_image_structure_empty()
dequantize.dequantize(image_data_quantized, image_data_dequantized)

In [8]:
### PAE ###
pae = Metrics(original_image, image_data_dequantized)
print(f"Pae: {pae.calculate_pae()}")
print(pae.PAE_data)


Pae: 9.0
9.0


In [9]:
### MSE ###
mse = Metrics(original_image, image_data_dequantized)
print(f"Mse: {mse.calculate_mse()}")

Mse: 28.00189208984375


In [10]:
### PSNR ###
psnr = Metrics(original_image, image_data_dequantized)
print(f"Psnr: {psnr.calculate_psnr()} dB")

Psnr: 33.69292552980915 dB


In [11]:
### S-transform forward ###
array = [1,2,3,4,8,6,10,8]
transform = Wavelet(array, 1)
array_transform = transform.s_tranform_forward(array)
print(f"Array modificado: {array_transform}")
    

Array modificado: [ 1  3  7  9 -1 -1  2  2]


In [12]:
### S-transform Inverse ###
array_inv = transform.s_transform_inverse(array_transform)
print(f"Array invertido: {array_inv}")


Array invertido: [ 1  2  3  4  8  6 10  8]


In [13]:
### S-transform Forward image ###
transform_image = Wavelet(original_image, 3)
#print(len(original_image[0][0]))
transform_forward = transform_image.handle_transform_forward()
print(transform_forward)


[[ 44  51  97 ...   3   3   3]
 [ 53 118 216 ...   4   5   4]
 [ 76 190 176 ...   5   5   6]
 ...
 [  4   4   5 ...   1   1   0]
 [  3   4   5 ...  -1   1   0]
 [  3   4   5 ...   1   1  -1]]


In [14]:
### S-transform Reverse image ###
transform_image_reverse = Wavelet(transform_forward, 3)
transform_reverse = transform_image_reverse.handle_transform_inverse()



In [15]:
print(transform_reverse)

[[[16 21 25 ... 25 22 19]
  [19 25 29 ... 29 25 22]
  [22 28 34 ... 34 29 25]
  ...
  [22 28 33 ... 34 30 26]
  [19 24 29 ... 30 25 23]
  [16 21 25 ... 25 22 19]]]


In [16]:
print(original_image)
 

[[[16 21 25 ... 25 22 19]
  [19 25 29 ... 29 25 22]
  [22 28 34 ... 34 29 25]
  ...
  [22 28 33 ... 34 30 26]
  [19 24 29 ... 30 25 23]
  [16 21 25 ... 25 22 19]]]


### Classe 17/09
Predicció:
X = 1,2,3,4,8,6,10,8
Causal
Pi = {0 si i=0, X(i-1) si i>0}
X'i = Xi - Pi
P = 0,1,2,3,4,8,6,10 (Predicció)
X' = 1,1,1,1,4,-2,4,-2
#
###### TODO
Predict sobre una matriu. -> ha de ser invertible -> img de 512 512
Calcular entropia del resultat X' -> H(x') <= H(x)

In [18]:
pred = Prediction(original_image)
mat_pred = pred.predict_image_LOP()
statistics_data_pred = statistics(mat_pred)
entropy_pred = statistics_data.compute_entropy()
print("Original entropy ",entropy)
print("Pred entropy: ", entropy_pred)

Original entropy  6.7004541529645625
Pred entropy:  20.101362458893682
